In [4]:
import random, csv, time, os, pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize
from scipy.stats import bernoulli
from scipy.io import savemat

%matplotlib inline

# Functions
 * **mle(w, pairs)**
 
 Calculate the MLE of w.
 
 Due to the fact that optimize.minimize only takes 1 input, here the pair information is imported from global variable.
 
 
 * **gradient(w,pairs)**
 
 Calculate the gradient of MLE. 
 
 Returns a numpy array with value of gradient.
 
 
 * **hessian(w, pairs)**
 
 Calculate the hessian of MLE. 
 
 Returns a numpy matrix with value of hessian.
 
 
 * **compare_rank(video_score, results, verbose=False, hist=False)**
 
 Compute the true rankings and rankings from results. 
 
 `verbose` will output with columns: Result Order, True Order, Result Score, Ture Score
    
 `hist` will output a histogram


 * **init_graph(num_nodes, num_edges, reference_matrix=None)**
 
 Marc's method of generating pairs
 
 
 * **matching_func(param, video_score, w_hat)**

 Function used to calculate L2 norm of v and w_star. Used to find a and b.
 Here param is [a, b]

    
 * **regularized_vector(video_score,w_hat)**

 Function to generate vector v after using matching_func to find optimal a and b
 
 
 
 * **performance_isabelle(video_score, video_num, w_hat)**

 Performance evaluation using method proposed by Isabelle.
 
 
 * **performance_nihar(video_score, video_num, w_hat)**

 Performance evaluation using method proposed by Nihar.
 
 
 * **noise_generation(video_score, pair)**

 Generate a decision of whether to flip, using proability from normal distribution and then Bernoulli to decide whether to flip
 
 
 * **sigmoid(x)**
 
 

In [5]:
def mle(w, pairs):    
    out = 1      
    for pair in pairs:
        if pair[0] == -1 or pair[1] == -1:
            continue
        out *= 1/(1+np.exp(-w[pair[0]] + w[pair[1]]))   
    return -np.log(out)

def gradient(w,pairs):
    grad = []
    for i in range(len(w)):
        gradient = 0

        for pair in pairs:
            if i == pair[0]:
                out = -1
            elif i == pair[1]:
                out = 1  
            else:
                continue
#             gradient -= out / (1/((np.exp(w[pair[1]]-w[pair[0]]))/2) +1 ) / 2
            gradient -= out / (1/((np.exp(w[pair[1]]-w[pair[0]]))) +1 )

        grad.append(-gradient)
        
    return np.array(grad)

def hessian(w, pairs):
    hessian = np.zeros((300, 300))
    for pair in pairs:
        exp_0 = np.exp(w[pair[1]]-w[pair[0]])
        hess = exp_0/(1+exp_0)**2
        hessian[pair[0], pair[1]] = hess
        hessian[pair[1], pair[0]] = hess
        
    return hessian

# def compare_rank(video_score, results, verbose=False, hist=False):
#     true_order = np.array(video_score).argsort()
#     true_ranks = true_order.argsort()

#     temp_o = np.array(results).argsort()
#     temp_r = temp_o.argsort()

#     resolution = 0.1
#     video_score_results = np.round(np.array(results)/resolution)*resolution
    
#     if verbose:
#         print 'Result Order \t True Order \t Result Score \t Ture Score'
#         for i in range(len(temp_r)):
#             print temp_r[i], '\t\t', true_ranks[i], '\t\t', video_score_results[i], '\t\t', video_score[i]
            
#     if hist:
#         diff = np.abs(temp_r - true_ranks)
#         plt.hist(diff, alpha=0.5)

        
def compare_rank(video_score, results, hist=False):
    scale_and_bias = np.dot(np.linalg.pinv(np.concatenate((results[:, None],
                                                           np.ones((len(results), 1))), axis=1)), video_score)

    difference = np.abs(video_score - (results * scale_and_bias[0] + scale_and_bias[1]))
    return difference
        

def init_graph(num_nodes, num_edges, reference_matrix=None):
    # If the number of edges is lower than num_nodes-1, raise error
    if num_edges < num_nodes:
        raise AttributeError('The number of edges must be >= num_nodes')

    # Define list of neighbors (with initial circular connection)
    neighbors = [[i-1, i+1] for i in range(num_nodes)]
    neighbors[0][0], neighbors[-1][1] = num_nodes-1, 0

    # Define list of node indices, maximum degree and number of placed nodes
    nodes, max_degree, num_placed = range(num_nodes), int(np.ceil((2.0 * num_edges) / num_nodes)), num_nodes

    def rewire_pairs(p0, p1):
        # Select existing pair
        ph1 = np.random.choice(num_nodes)
        ph2 = neighbors[ph1][np.random.choice(len(neighbors[ph1]))]
        while p0 in neighbors[ph2] or p1 in neighbors[ph1] or p0 in [ph1, ph2] or p1 in [ph1, ph2]:
            ph1 = np.random.choice(num_nodes)
            ph2 = neighbors[ph1][np.random.choice(len(neighbors[ph1]))]

        # Rewire with current pair
        neighbors[ph1].remove(ph2)
        neighbors[ph2].remove(ph1)
        neighbors[ph1].append(p1)
        neighbors[p1].append(ph1)
        neighbors[ph2].append(p0)
        neighbors[p0].append(ph2)

    # Start placing random edges
    ct = 0
    while num_placed < num_edges:
        # If only one node left, rewire with existing pair
        if len(nodes) == 1:
            rewire_pairs(nodes[0], nodes[0])
            num_placed += 1
            continue

        # Sample random pair of nodes
        pair = np.random.choice(nodes, size=2, replace=False)

        # If pair already connected, do rewiring
        if pair[0] in neighbors[pair[1]]:
            if ct < 100:
                ct += 1
                continue
            rewire_pairs(pair[0], pair[1])
            num_placed += 1

        # Else, place edge between nodes
        else:
            neighbors[pair[0]].append(pair[1])
            neighbors[pair[1]].append(pair[0])
            num_placed += 1

        ct = 0

        # If max degree reached for first node, remove from sampling list
        if len(neighbors[pair[0]]) >= max_degree:
            nodes.remove(pair[0])

        # If max degree reached for second node, remove from sampling list
        if len(neighbors[pair[1]]) >= max_degree:
            nodes.remove(pair[1])

    # Return pairs
    return [(
        (i, j) if reference_matrix is None or reference_matrix[i] > reference_matrix[j] else (j, i)
    ) for i, il in enumerate(neighbors) for j in sorted(il) if i < j]


def matching_func(param, video_score, w_hat):
    return np.linalg.norm(video_score - param[0]*np.array(w_hat) - param[1])


def regularized_vector(video_score,w_hat):
    coeff = optimize.minimize(matching_func, [0, 0], args=(video_score, w_hat))

    a = coeff['x'][0]
    b = coeff['x'][1]
    v = a*np.array(w_hat)+b
    return v

def performance_isabelle(video_score, video_num, w_hat):
    epsilon = 0.0001
    error = 0
    v = regularized_vector(video_score,w_hat)

    true_order = np.array(video_score).argsort()

    what = [ w_hat[i] for i in true_order]
    wstar = [ video_score[i] for i in true_order]

    for i in range(video_num-2):
        error += np.abs((wstar[i+1]-wstar[i])/(wstar[i+2]-wstar[i]+epsilon)-
                        (what[i+1]-what[i])/(what[i+2]-what[i]+epsilon))
    
    return error

def performance_nihar(video_score, w_hat):
    v = regularized_vector(video_score,w_hat)
    return np.linalg.norm(video_score - v)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def noise_generation(video_score, pair):
    difference = video_score[pair[0]] - video_score[pair[1]]
    p = sigmoid(difference)*(1-sigmoid(difference))
    decision = bernoulli.rvs(p,size=1)

    return decision

# Data Generation
The cell below generates data for $video\_num$ videos, calculates all possible pairs and stores in $pairs\_truth$ and $total_pairs$ is the number of pairs. 

In [26]:
video_nums = [5000,10000]
# num_edges_pct = [0.01,0.1,1,10,100]

# video_nums = [10000]
num_edges_pct = [0.1, 0.5, 1, 2.17, 3]

for iteration in range(1):
    for video_num in video_nums:
        for num_edge_pct in num_edges_pct:

            num_edge = int(num_edge_pct*video_num*np.log(video_num))

            if num_edge < video_num:
                num_edge = video_num
            if num_edge > video_num*(video_num-1)/2:
                num_edge = video_num*(video_num-1)/2

            video_score = np.random.uniform(-5,5,video_num)
            resolution = 0.1
            video_score = np.round(video_score/resolution)*resolution

            w = np.ones(video_num)
            test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs),num_edge)]
            test_pairs = init_graph(video_num, num_edge, video_score)
            for i in range(len(test_pairs)):
                flip = noise_generation(video_score, test_pairs[i])
                if flip:
                    test_pairs[i] = (test_pairs[i][1], test_pairs[i][0])
            start_time = time.time()

            res = optimize.minimize(mle, w, 
                                    method='Newton-CG',
                                    jac=gradient,
                                    args=(test_pairs,),
                                    tol = 10,
                                    options={'disp': False})

            filename = 'result/random_error_'+ str(video_num)+'_at_'+str(num_edge)+'_iter_00'+str(iteration)+'.p'
            pickle.dump( dict({'x':res.x, 'video_score':video_score}), open( filename, "wb" ) )
            print  'Time on Iteration %d with Video Number %d and edges %.2f : %.1f seconds' %(iteration, video_num, num_edge, float(time.time() - start_time))

Time on Iteration 0 with Video Number 625 and edges 625.00 : 0.3 seconds
Time on Iteration 0 with Video Number 625 and edges 2011.00 : 1.5 seconds
Time on Iteration 0 with Video Number 625 and edges 4023.00 : 3.1 seconds
Time on Iteration 0 with Video Number 625 and edges 8731.00 : 7.6 seconds
Time on Iteration 0 with Video Number 625 and edges 12070.00 : 9.6 seconds
Time on Iteration 0 with Video Number 1250 and edges 1250.00 : 1.3 seconds
Time on Iteration 0 with Video Number 1250 and edges 4456.00 : 6.3 seconds
Time on Iteration 0 with Video Number 1250 and edges 8913.00 : 13.4 seconds
Time on Iteration 0 with Video Number 1250 and edges 19342.00 : 31.7 seconds
Time on Iteration 0 with Video Number 1250 and edges 26740.00 : 47.2 seconds
Time on Iteration 0 with Video Number 2500 and edges 2500.00 : 5.0 seconds
Time on Iteration 0 with Video Number 2500 and edges 9780.00 : 31.8 seconds
Time on Iteration 0 with Video Number 2500 and edges 19560.00 : 66.6 seconds
Time on Iteration 0 wi

In [6]:
video_nums = [5000,10000]

num_edges_pct = [0.1, 0.5, 1, 2.17, 3]

for iteration in range(1):
    for video_num in video_nums:
        for num_edge_pct in num_edges_pct:

            num_edge = int(num_edge_pct*video_num*np.log(video_num))

            if num_edge < video_num:
                num_edge = video_num
            if num_edge > video_num*(video_num-1)/2:
                num_edge = video_num*(video_num-1)/2

            video_score = np.random.uniform(-5,5,video_num)
            resolution = 0.1
            video_score = np.round(video_score/resolution)*resolution
            
            pairs_truth = []
            
            for i in range(len(video_score)-1):
                for j in range(i+1, len(video_score)):
                    if video_score[i] > video_score[j]:
                        pairs_truth.append((i,j))
                    else:
                        pairs_truth.append((j,i))

            total_pairs = len(pairs_truth)
            
            w = np.ones(video_num)
            
            test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs),num_edge)]
            
            for i in range(len(test_pairs)):
                flip = noise_generation(video_score, test_pairs[i])
                if flip:
                    test_pairs[i] = (test_pairs[i][1], test_pairs[i][0])
                    
            start_time = time.time()

            res = optimize.minimize(mle, w, 
                                    method='Newton-CG',
                                    jac=gradient,
                                    args=(test_pairs,),
                                    tol = 10,
                                    options={'disp': False})

            filename = 'result/random_error_'+ str(video_num)+'_at_'+str(num_edge)+'.p'
            pickle.dump( dict({'x':res.x, 'video_score':video_score}), open( filename, "wb" ) )
            print  'Time on Iteration %d with Video Number %d and edges %.2f : %.1f seconds' %(iteration, video_num, num_edge, float(time.time() - start_time))

Time on Iteration 0 with Video Number 5000 and edges 5000.00 : 29.1 seconds
Time on Iteration 0 with Video Number 5000 and edges 21292.00 : 201.8 seconds
Time on Iteration 0 with Video Number 5000 and edges 42585.00 : 407.4 seconds
Time on Iteration 0 with Video Number 5000 and edges 92411.00 : 839.9 seconds
Time on Iteration 0 with Video Number 5000 and edges 127757.00 : 1226.2 seconds
Time on Iteration 0 with Video Number 10000 and edges 10000.00 : 207.4 seconds
Time on Iteration 0 with Video Number 10000 and edges 46051.00 : 999.9 seconds
Time on Iteration 0 with Video Number 10000 and edges 92103.00 : 2041.0 seconds
Time on Iteration 0 with Video Number 10000 and edges 199864.00 : 4695.4 seconds


KeyboardInterrupt: 

In [7]:
video_nums = [10000]

num_edges_pct = [3]

for iteration in range(1):
    for video_num in video_nums:
        for num_edge_pct in num_edges_pct:

            num_edge = int(num_edge_pct*video_num*np.log(video_num))

            if num_edge < video_num:
                num_edge = video_num
            if num_edge > video_num*(video_num-1)/2:
                num_edge = video_num*(video_num-1)/2

            video_score = np.random.uniform(-5,5,video_num)
            resolution = 0.1
            video_score = np.round(video_score/resolution)*resolution
            
            pairs_truth = []
            
            for i in range(len(video_score)-1):
                for j in range(i+1, len(video_score)):
                    if video_score[i] > video_score[j]:
                        pairs_truth.append((i,j))
                    else:
                        pairs_truth.append((j,i))

            total_pairs = len(pairs_truth)
            
            w = np.ones(video_num)
            
            test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs),num_edge)]
            
            for i in range(len(test_pairs)):
                flip = noise_generation(video_score, test_pairs[i])
                if flip:
                    test_pairs[i] = (test_pairs[i][1], test_pairs[i][0])
                    
            start_time = time.time()

            res = optimize.minimize(mle, w, 
                                    method='Newton-CG',
                                    jac=gradient,
                                    args=(test_pairs,),
                                    tol = 10,
                                    options={'disp': False})

            filename = 'result/random_error_'+ str(video_num)+'_at_'+str(num_edge)+'.p'
            pickle.dump( dict({'x':res.x, 'video_score':video_score}), open( filename, "wb" ) )
            print  'Time on Iteration %d with Video Number %d and edges %.2f : %.1f seconds' %(iteration, video_num, num_edge, float(time.time() - start_time))

Time on Iteration 0 with Video Number 10000 and edges 276310.00 : 7812.3 seconds


In [20]:
print test_pairs

[(0, 1), (0, 7), (0, 9), (1, 2), (1, 8), (2, 3), (2, 4), (3, 4), (3, 7), (4, 5), (5, 6), (5, 9), (6, 7), (6, 8), (8, 9)]


In [24]:
for pair in test_pairs[0:5]:
    pair = (pair[1],pair[1])
print test_pairs

[(0, 1), (0, 7), (0, 9), (1, 2), (1, 8), (2, 3), (2, 4), (3, 4), (3, 7), (4, 5), (5, 6), (5, 9), (6, 7), (6, 8), (8, 9)]
